# Procesar el txt con spacy-stanza y guardar en un fichero binario: A_Procesar_Guardar.ipynb
Este es el primer *notebook* para procesar el censo de linajes asturianos de García Linares. Si dispone del censo en txt, escriba la dirección al mismo en la variable `direcciontxt`. De lo contrario, continúe usando el pequeño extracto del censo guardado en la dirección `./Documentos/PadronOCRPrueba.txt`.

En este *notebook* de *Python* vamos a realizar las siguientes tareas:
- Cargamos un documento txt. El `txt` se encuentra en la dirección `direcciontxt`.
- Cargamos los módulos `spacy` y `spacy-stanza`. El primero nos ofrece toda la funcionalidad y facilidad de tratamiento de Spacy. El segundo brinda toda la exactitud en los resultados del procesamiento con Stanza. Recuerde que debe tener instalado las librerías de Stanza, desarrollado por la Universidad de Stanford.
- Guardamos el documento procesado como un fichero de datos (documento binario). El binario lo guardaremos en la dirección `direccionbin`. De esta forma no tenemos que procesar el documento cada vez que abrimos *Jupyter Lab*.

## Importamos las librerías a usar
Lo haremos siempre al comienzo de cada *notebook*.

In [ ]:
import spacy
import spacy_stanza
from spacy.tokens import Doc
from spacy.tokens import DocBin

# Ejecutamos nuestras funciones creadas para la ocasión. Están en el notebook Z_Funciones.ipynb.
%run './Z_Funciones.ipynb'

## Especificamos las direcciones y configuración del pipeline
Esta sección es la que podemos personalizar o modificar a conveniencia según el archivo que queramos procesar, dónde lo queremos guardar, etc. Contiene los siguientes datos:
- `direcciontxt`: es la dirección del archivo de txt que vamos a procesar con Spacy-Stanza. Nota: cada línea del txt debe ser una frase completa limitada por un punto.
- `direcciontxtprueba`: es la dirección de una pequeña parte archivo de txt que vamos a procesar con Spacy-Stanza. Es un archivo más pequeño que podemos usar para pruebas. Nota: cada línea del txt debe ser una frase completa limitada por un punto.
- `direccionbin`: es la dirección donde vamos a guardar el archivo procesado por Spacy-Stanza. Es un archivo binario.
- `nlpconfig`: es la variable que determina las características del pipeline de procesamiento de spacy-stanza. Por ejemplo, contiene el idioma, cuántas frases procesamos de una sola vez, etc.

In [ ]:
direcciontxt = './Documentos/PadronOCRPrueba.txt'
direccionbin = './Documentos/Padronbin'
# direcciontxt = './Privado/PadronOCR.txt'
# direccionbin = './Privado/PadronbinTOTAL'
nlpconfig = {
    'name': 'es', # Language code for the language to build the Pipeline in
    'tokenize_batch_size': 32, # Enseguida vamos a cambiarlo por la cantidad máxima de palabras que alberga una frase en nuestro documento.
    'ner_batch_size': 32 # Enseguida vamos a cambiarlo por la cantidad máxima de palabras que alberga una frase en nuestro documento.
}

## Cargamos el archivo de texto
Vamos a convertir el archivo de texto a una variable de Python que Spacy-Stanza pueda procesar. La variable se llamará `lineas` porque es una lista que contiene cada una de las lineas del txt.

In [ ]:
# Cargamos el texto
with open(direcciontxt, encoding='utf-8') as f:
    
    # Creamos la variable 'lineas' como una lista vacía que vamos rellenando.
    lineas = []
    for linea in f:
        lineas.append(linea.strip())

# Mostremos las primeras 15 líneas.
print(lineas[0:15])

Vamos a corregir el número máximo de palabras que el pipeline para resolver el NER. Vamos a hacer que sea igual al número máximo de palabras que tienen las frases de nuestro documento, pero aumentado un 25% para tener en cuenta los tokens de puntuación. Usaremos la función `cuenta_palabras_max`.

In [ ]:
# Contamos las palabras y corregimos el diccionario nlpconfig
numero1 = cuenta_palabras_max(direcciontxt)
numero2 = int(numero1 * 1.25)

nlpconfig['ner_batch_size'] = numero2
nlpconfig['tokenize_batch_size'] = numero2

print("El número de palabras máximas es", str(numero1) + ".", "\nAñadiendo un 25%, tomaremos el número", str(numero2) + ".")

## Cargamos el pipeline de procesamiento de spacy-stanza

Creamos el pipeline (la función que procesa nuestro archivo de análisis).

In [ ]:
#Cargamos el pipeline de spacy-stanza para resolver el problema NER.
nlp = spacy_stanza.load_pipeline(**nlpconfig)

## Aplicamos el pipeline de spacy-stanza
Aplicamos el pipeline a nuestro modelo: 
- En primer lugar, `nlp.pipe` crea un iterable de frases a ser procesadas una a una.
- Al ejecutar la orden `list`, no solo estamos convirtiendo el iterable en una lista, sino que también se aplica, frase a frase, la función `nlp`. 
- Finalmente, unimos cada frase en un solo documento de SpaCy llamado `doc`.

In [ ]:
doc = Doc.from_docs(list(nlp.pipe(lineas)))

# Serializamos
La palabra serializar se refiere a guardar nuestro documento procesado `doc` a un archivo binario que se pueda alojar en un disco duro. Así no tenemos que procesar el documento de texto cada vez que abrimos *Jupyter Lab*. Guardamos el binario en la dirección `direccionbin`. Para ello usaremos la función `serializacion` definida en el archivo *funciones_Spacy.ipynb*.

In [ ]:
# Serializamos
serializacion(doc, direccionbin)